In [148]:
# Imports:
import pandas as pd
import numpy as np
# non-library stuff
from utils import *

# Retrieve Data
data = retrieve_data()
train = data['train']
test = data['test']
train_num = data['train_num']
y_feature = 'SalePrice'

1. Preprocess:
    - Impute missing numerical values
    - Encode categorical variables 
    - Try to accordingly (to their discrption) preprocess data
2. Find the best features by running linear regression on the different combinations of data

In [149]:
# LotFrontage and GarageYrBlt should equal zeos 
# for NaN's since it means 0 meters of lot frontage
train['LotFrontage'] = train['LotFrontage'].fillna(0)
train['MasVnrArea'] = train['MasVnrArea'].fillna(0)
train['GarageYrBlt'] = fillNaWithKNN('GarageYrBlt', data['train'].copy(), 'SalePrice')

# The missing numerical features
# data['train_num_missing']

# Cheking to see if the imputations have worked or not
True in train[data['train_num_list']].isna().any()

False

In [ ]:
# use this structure for encoding categorical data
# dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [150]:
train_dataset = train.copy()[train.corr()['SalePrice'].nlargest(21)[1:].keys().to_list()]
train_labels = train.copy().pop("SalePrice")

X = train_dataset.copy()
mean_X = normalize(train_dataset.copy(), Type='std')
std_X = normalize(train_dataset.copy())

mean_y = normalize(train_labels.copy())
std_y = normalize(train_labels.copy(), Type='std')
sc_y = train_labels.copy() / 10000
y = train_labels.copy()

In [89]:
train_labels.head(15)

0     208500
1     181500
2     223500
3     140000
4     250000
5     143000
6     307000
7     200000
8     129900
9     118000
10    129500
11    345000
12    144000
13    279500
14    157000
Name: SalePrice, dtype: int64

In [87]:
normalize(train_labels.copy()).head(15)

0    -0.347154
1    -0.007286
2    -0.535970
3     0.515105
4    -0.869545
5     0.477341
6    -1.587045
7    -0.240159
8     0.642241
9     0.792034
10    0.647276
11   -2.065378
12    0.464754
13   -1.240882
14    0.301113
Name: SalePrice, dtype: float64

In [86]:
normalize(train_labels.copy(), Type='std').head(15)

0      -0.152435
1      -0.003199
2      -0.235344
3       0.226182
4      -0.381817
          ...   
1455    0.032728
1456   -0.160726
1457   -0.473017
1458    0.214437
1459    0.184728
Name: SalePrice, Length: 1460, dtype: float64

In [151]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, input_shape=[len(std_d.keys())]),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='softmax'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='softmax'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='softmax'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.01)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse', 'msle'])
  return model

model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=110)

In [152]:
EPOCHS = 2000
batch_size = 150
val_split = 0.3

In [154]:
history = model.fit(std_d, std_y, batch_size=batch_size,
                    epochs=EPOCHS, validation_split =val_split, verbose=0, 
                    callbacks=[tfdocs.modeling.EpochDots()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist


Epoch: 0, loss:0.0269,  mse:0.0269,  msle:0.0039,  val_loss:0.0465,  val_mse:0.0465,  val_msle:0.0068,  
....................................................................................................
Epoch: 100, loss:0.0259,  mse:0.0259,  msle:0.0034,  val_loss:0.0502,  val_mse:0.0502,  val_msle:0.0050,  
....................................................................................................
Epoch: 200, loss:0.0246,  mse:0.0246,  msle:0.0031,  val_loss:0.0397,  val_mse:0.0397,  val_msle:0.0043,  
....................................................................................................
Epoch: 300, loss:0.0237,  mse:0.0237,  msle:0.0033,  val_loss:0.0365,  val_mse:0.0365,  val_msle:0.0061,  
....................................................................................................
Epoch: 400, loss:0.0184,  mse:0.0184,  msle:0.0036,  val_loss:0.0346,  val_mse:0.0346,  val_msle:0.0050,  
..............................................................

,loss,mse,msle,val_loss,val_mse,val_msle,epoch
0,0.026947,0.026947,0.003868,0.046461,0.046461,0.006827,0
1,0.018165,0.018165,0.003572,0.030614,0.030614,0.005155,1
2,0.017248,0.017248,0.003269,0.037457,0.037457,0.005428,2
3,0.019905,0.019905,0.003527,0.046125,0.046125,0.004970,3
4,0.020388,0.020388,0.003577,0.050908,0.050908,0.006744,4
...,...,...,...,...,...,...,...
1995,0.026865,0.026865,0.003806,0.032266,0.032266,0.004431,1995
1996,0.022157,0.022157,0.003268,0.032370,0.032370,0.004983,1996
1997,0.024654,0.024654,0.003016,0.037132,0.037132,0.004555,1997
1998,0.018365,0.018365,0.003387,0.039673,0.039673,0.005916,1998


In [146]:
history = model.fit(std_X, y, batch_size=batch_size,
                    epochs=EPOCHS, validation_split =val_split, verbose=0, 
                    callbacks=[tfdocs.modeling.EpochDots()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist


Epoch: 0, loss:39391375360.0000,  mae:181699.6406,  mse:39391375360.0000,  msle:125.6064,  val_loss:38217502720.0000,  val_mae:179102.8281,  val_mse:38217506816.0000,  val_msle:107.6575,  
....................................................................................................
Epoch: 100, loss:38206935040.0000,  mae:178349.0469,  mse:38206935040.0000,  msle:16.9415,  val_loss:37021835264.0000,  val_mae:175733.2188,  val_mse:37021835264.0000,  val_msle:15.3115,  
....................................................................................................
Epoch: 200, loss:35067932672.0000,  mae:169219.7031,  mse:35067932672.0000,  msle:7.9732,  val_loss:33906849792.0000,  val_mae:166634.8281,  val_mse:33906849792.0000,  val_msle:6.8397,  
....................................................................................................
Epoch: 300, loss:30440126464.0000,  mae:154492.4688,  mse:30440126464.0000,  msle:4.3023,  val_loss:29203982336.0000,  val_mae:1518

,loss,mae,mse,msle,val_loss,val_mae,val_mse,val_msle,epoch
0,3.939138e+10,181699.640625,3.939138e+10,125.606361,3.821750e+10,179102.828125,3.821751e+10,107.657494,0
1,3.938953e+10,181694.515625,3.938953e+10,102.580086,3.821539e+10,179096.968750,3.821539e+10,92.474373,1
2,3.938747e+10,181688.828125,3.938747e+10,93.143150,3.821341e+10,179091.390625,3.821341e+10,84.887863,2
3,3.938509e+10,181682.328125,3.938509e+10,85.559875,3.821124e+10,179085.343750,3.821124e+10,79.289246,3
4,3.938308e+10,181676.640625,3.938308e+10,80.949043,3.820898e+10,179079.031250,3.820898e+10,74.974953,4
...,...,...,...,...,...,...,...,...,...
1995,6.377791e+09,58348.175781,6.377791e+09,0.171197,6.146023e+09,55933.070312,6.146023e+09,0.156790,1995
1996,6.376897e+09,58334.566406,6.376897e+09,0.171126,6.146116e+09,55937.937500,6.146116e+09,0.156809,1996
1997,6.376705e+09,58340.144531,6.376705e+09,0.171134,6.146102e+09,55937.203125,6.146102e+09,0.156806,1997
1998,6.376375e+09,58312.105469,6.376375e+09,0.171053,6.146375e+09,55951.371094,6.146375e+09,0.156860,1998


In [158]:
dir(history)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_chief_worker_only',
 '_implements_predict_batch_hooks',
 '_implements_test_batch_hooks',
 '_implements_train_batch_hooks',
 '_keras_api_names',
 '_keras_api_names_v1',
 'epoch',
 'history',
 'model',
 'on_batch_begin',
 'on_batch_end',
 'on_epoch_begin',
 'on_epoch_end',
 'on_predict_batch_begin',
 'on_predict_batch_end',
 'on_predict_begin',
 'on_predict_end',
 'on_test_batch_begin',
 'on_test_batch_end',
 'on_test_begin',
 'on_test_end',
 'on_train_batch_begin',
 'on_train_batch_end',
 'on_train_begin',
 'on_train_end',
 'params',
 'set_model',
 'set_params',
 'validation_data']

In [161]:
dir(history.model)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_inbound_node',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_attribute_sentinel',
 '_auto_track_sub_layers',
 '_autocast',
 '_base_init',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_collect_input_masks',
 '_compile_was_called',
 '_compiled_trainable_state',
 '_compute_dtype',
 '_co